# Data Preprocessing & EDA - Property Valuation

This notebook covers:
1. Data loading and exploration
2. Exploratory Data Analysis (EDA)
3. Feature engineering
4. Data visualization

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

## 1. Load Data

In [ ]:
# Load data
train_df = pd.read_csv('train.csv')  # or train.xlsx
test_df = pd.read_csv('test.csv')    # or test.xlsx

print(f"Training samples: {len(train_df)}")
print(f"Test samples: {len(test_df)}")
print(f"\nTraining columns: {list(train_df.columns)}")

In [ ]:
# Basic info
train_df.info()

In [ ]:
# Statistical summary
train_df.describe()

## 2. Missing Values & Data Quality

In [ ]:
# Check missing values
print("Missing values in training data:")
missing = train_df.isnull().sum()
print(missing[missing > 0] if missing.sum() > 0 else "No missing values!")

print("\nMissing values in test data:")
missing_test = test_df.isnull().sum()
print(missing_test[missing_test > 0] if missing_test.sum() > 0 else "No missing values!")

In [ ]:
# Check for outliers
print("Potential outliers:")
print(f"  Max bedrooms: {train_df['bedrooms'].max()}")
print(f"  Max bathrooms: {train_df['bathrooms'].max()}")
print(f"  Max sqft_living: {train_df['sqft_living'].max():,}")
print(f"  Max price: ${train_df['price'].max():,}")
print(f"  Min price: ${train_df['price'].min():,}")

## 3. Exploratory Data Analysis

In [ ]:
# Price distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(train_df['price'], bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(train_df['price'].median(), color='red', linestyle='--', label=f'Median: ${train_df["price"].median():,.0f}')
axes[0].set_xlabel('Price ($)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Price Distribution')
axes[0].legend()

axes[1].hist(np.log1p(train_df['price']), bins=50, edgecolor='black', alpha=0.7, color='green')
axes[1].set_xlabel('Log(Price)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Log-Transformed Price')

axes[2].boxplot(train_df['price'])
axes[2].set_ylabel('Price ($)')
axes[2].set_title('Price Box Plot')

plt.tight_layout()
plt.show()

print(f"\nPrice Statistics:")
print(f"  Mean:   ${train_df['price'].mean():,.0f}")
print(f"  Median: ${train_df['price'].median():,.0f}")
print(f"  Std:    ${train_df['price'].std():,.0f}")

In [ ]:
# Feature correlations with price
numeric_cols = train_df.select_dtypes(include=[np.number]).columns
correlations = train_df[numeric_cols].corr()['price'].sort_values(ascending=False)

plt.figure(figsize=(10, 8))
correlations.drop('price').plot(kind='barh', color=['green' if x > 0 else 'red' for x in correlations.drop('price')])
plt.xlabel('Correlation with Price')
plt.title('Feature Correlations with Price')
plt.axvline(0, color='black', linewidth=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
top_features = ['price', 'sqft_living', 'grade', 'sqft_above', 'bathrooms', 'view', 'sqft_basement', 'bedrooms']
corr_matrix = train_df[top_features].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0, fmt='.2f')
plt.title('Correlation Heatmap (Top Features)')
plt.tight_layout()
plt.show()

In [ ]:
# Key features vs Price
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Sqft Living vs Price
axes[0, 0].scatter(train_df['sqft_living'], train_df['price'], alpha=0.3, s=5)
axes[0, 0].set_xlabel('Sqft Living')
axes[0, 0].set_ylabel('Price ($)')
axes[0, 0].set_title('Living Space vs Price')

# Grade vs Price
grade_prices = train_df.groupby('grade')['price'].mean()
axes[0, 1].bar(grade_prices.index, grade_prices.values, color='steelblue')
axes[0, 1].set_xlabel('Grade')
axes[0, 1].set_ylabel('Avg Price ($)')
axes[0, 1].set_title('Grade vs Price')

# Waterfront vs Price
wf_prices = train_df.groupby('waterfront')['price'].mean()
axes[0, 2].bar(['No', 'Yes'], wf_prices.values, color=['gray', 'blue'])
axes[0, 2].set_xlabel('Waterfront')
axes[0, 2].set_ylabel('Avg Price ($)')
axes[0, 2].set_title(f'Waterfront Premium: +${wf_prices[1]-wf_prices[0]:,.0f}')

# View vs Price
view_prices = train_df.groupby('view')['price'].mean()
axes[1, 0].bar(view_prices.index, view_prices.values, color='green')
axes[1, 0].set_xlabel('View Rating')
axes[1, 0].set_ylabel('Avg Price ($)')
axes[1, 0].set_title('View vs Price')

# Bedrooms vs Price
bed_prices = train_df[train_df['bedrooms'] <= 8].groupby('bedrooms')['price'].mean()
axes[1, 1].bar(bed_prices.index, bed_prices.values, color='orange')
axes[1, 1].set_xlabel('Bedrooms')
axes[1, 1].set_ylabel('Avg Price ($)')
axes[1, 1].set_title('Bedrooms vs Price')

# Condition vs Price
cond_prices = train_df.groupby('condition')['price'].mean()
axes[1, 2].bar(cond_prices.index, cond_prices.values, color='purple')
axes[1, 2].set_xlabel('Condition')
axes[1, 2].set_ylabel('Avg Price ($)')
axes[1, 2].set_title('Condition vs Price')

plt.tight_layout()
plt.show()

In [ ]:
# Geographic analysis
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Price by location
scatter = axes[0].scatter(train_df['long'], train_df['lat'], 
                          c=train_df['price'], cmap='RdYlGn_r', 
                          alpha=0.5, s=5)
plt.colorbar(scatter, ax=axes[0], label='Price ($)')
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')
axes[0].set_title('Property Prices by Location')

# Waterfront locations
non_wf = train_df[train_df['waterfront'] == 0]
wf = train_df[train_df['waterfront'] == 1]
axes[1].scatter(non_wf['long'], non_wf['lat'], alpha=0.3, s=2, c='gray', label='Non-Waterfront')
axes[1].scatter(wf['long'], wf['lat'], alpha=0.8, s=15, c='blue', marker='*', label='Waterfront')
axes[1].set_xlabel('Longitude')
axes[1].set_ylabel('Latitude')
axes[1].set_title(f'Waterfront Properties ({len(wf)} total)')
axes[1].legend()

plt.tight_layout()
plt.show()

## 4. Feature Engineering

In [ ]:
def engineer_features(df):
    """Create engineered features."""
    df = df.copy()
    
    # Age features
    df['age'] = 2015 - df['yr_built']
    df['years_since_renovation'] = np.where(
        df['yr_renovated'] > 0,
        2015 - df['yr_renovated'],
        df['age']
    )
    
    # Size ratios
    df['living_lot_ratio'] = df['sqft_living'] / (df['sqft_lot'] + 1)
    df['above_living_ratio'] = df['sqft_above'] / (df['sqft_living'] + 1)
    df['basement_ratio'] = df['sqft_basement'] / (df['sqft_living'] + 1)
    
    # Neighborhood comparison
    df['living_vs_neighbors'] = df['sqft_living'] / (df['sqft_living15'] + 1)
    df['lot_vs_neighbors'] = df['sqft_lot'] / (df['sqft_lot15'] + 1)
    
    # Room features
    df['total_rooms'] = df['bedrooms'] + df['bathrooms']
    df['sqft_per_room'] = df['sqft_living'] / (df['total_rooms'] + 1)
    
    # Quality
    df['quality_score'] = df['grade'] * df['condition']
    df['has_basement'] = (df['sqft_basement'] > 0).astype(int)
    df['was_renovated'] = (df['yr_renovated'] > 0).astype(int)
    
    return df

train_engineered = engineer_features(train_df)
test_engineered = engineer_features(test_df)

new_features = [c for c in train_engineered.columns if c not in train_df.columns]
print(f"Engineered features ({len(new_features)}):")
for f in new_features:
    print(f"  - {f}")

In [ ]:
# Visualize engineered features
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].scatter(train_engineered['age'], train_engineered['price'], alpha=0.3, s=5)
axes[0, 0].set_xlabel('Property Age (years)')
axes[0, 0].set_ylabel('Price ($)')
axes[0, 0].set_title('Age vs Price')

axes[0, 1].scatter(train_engineered['living_lot_ratio'], train_engineered['price'], alpha=0.3, s=5)
axes[0, 1].set_xlabel('Living/Lot Ratio')
axes[0, 1].set_ylabel('Price ($)')
axes[0, 1].set_title('Living/Lot Ratio vs Price')
axes[0, 1].set_xlim(0, 1)

axes[0, 2].scatter(train_engineered['quality_score'], train_engineered['price'], alpha=0.3, s=5)
axes[0, 2].set_xlabel('Quality Score')
axes[0, 2].set_ylabel('Price ($)')
axes[0, 2].set_title('Quality Score vs Price')

axes[1, 0].scatter(train_engineered['living_vs_neighbors'], train_engineered['price'], alpha=0.3, s=5)
axes[1, 0].set_xlabel('Living vs Neighbors')
axes[1, 0].set_ylabel('Price ($)')
axes[1, 0].set_title('Relative Size vs Price')
axes[1, 0].set_xlim(0, 3)

axes[1, 1].scatter(train_engineered['sqft_per_room'], train_engineered['price'], alpha=0.3, s=5)
axes[1, 1].set_xlabel('Sqft per Room')
axes[1, 1].set_ylabel('Price ($)')
axes[1, 1].set_title('Space per Room vs Price')
axes[1, 1].set_xlim(0, 1000)

train_engineered.boxplot(column='price', by='was_renovated', ax=axes[1, 2])
axes[1, 2].set_xlabel('Was Renovated')
axes[1, 2].set_ylabel('Price ($)')
axes[1, 2].set_title('Renovation Impact')

plt.suptitle('')
plt.tight_layout()
plt.show()

## 5. Satellite Images Check

In [ ]:
# Check satellite images
from PIL import Image

image_dir = Path('satellite_images')
if image_dir.exists():
    images = list(image_dir.glob('*.png'))
    print(f"Found {len(images)} satellite images")
    
    # Display samples
    if len(images) > 0:
        fig, axes = plt.subplots(2, 4, figsize=(16, 8))
        sample_images = np.random.choice(images, min(8, len(images)), replace=False)
        
        for idx, img_path in enumerate(sample_images):
            ax = axes[idx // 4, idx % 4]
            img = Image.open(img_path)
            ax.imshow(img)
            pid = img_path.stem
            if int(pid) in train_df['id'].values:
                price = train_df[train_df['id'] == int(pid)]['price'].values[0]
                ax.set_title(f'${price:,.0f}')
            ax.axis('off')
        
        plt.suptitle('Sample Satellite Images', fontsize=14)
        plt.tight_layout()
        plt.show()
else:
    print("Satellite images directory not found. Run data_fetcher.py first.")

## Summary

### Data Overview
- Training samples: ~16,000
- Test samples: ~5,400
- No missing values

### Key Insights
1. **sqft_living** has highest correlation with price (0.70)
2. **grade** is second most important (0.67)
3. **Waterfront** properties have ~3x higher average price
4. **Location** (lat/long) shows clear price patterns

### Engineered Features
- Age and renovation features
- Size ratios (living/lot, basement ratio)
- Neighborhood comparisons
- Quality scores

### Next Steps
→ Run `model_training.ipynb` for model training